In [1]:
!pip install pysrt tqdm pprint

In [2]:
!pip install -e git+https://github.com/yoyonel/py-googletrans.git#egg=googletrans

Obtaining googletrans from git+https://github.com/yoyonel/py-googletrans.git#egg=googletrans
  Updating /home/latty/.virtualenvs/py3.6_pydbsrt/src/googletrans clone
  Found existing installation: googletrans 2.3.0
    Uninstalling googletrans-2.3.0:
      Successfully uninstalled googletrans-2.3.0
  Running setup.py develop for googletrans


In [3]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [4]:
srt_fr_path = "The Golden Child.DVD_PAL.fre.srt"
srt_en_path = "The Golden Child (1986) 1080p web.en.srt"

In [5]:
import pysrt

srt_fr = pysrt.open(srt_fr_path)
srt_en = pysrt.open(srt_en_path)

In [14]:
srt_en[0].text, srt_fr[2].text

("Don't let him touch you!", 'Qu’il ne vous touche pas.')

In [15]:
srt_fr.shift(ratio=25.00/23.9)

In [16]:
diff_starts = srt_fr[2].start - srt_en[0].start
print(diff_starts)
#srt_fr.shift(seconds=diff_starts.seconds, milliseconds=diff_starts.milliseconds)

00:00:13,217


In [17]:
[
    (srt_fr[i+2].text, srt_en[i].text, srt_fr[i+2].start - srt_en[i].start)
    for i in range(100)
]

[('Qu’il ne vous touche pas.',
  "Don't let him touch you!",
  SubRipTime(0, 0, 13, 217)),
 ('<i>Tartes aux poils.</i>', '<i>Butt Pie.</i>', SubRipTime(0, 0, 19, 878)),
 ('<i>Gâteaux aux poils,</i>\nc’est la suite d’un bouquin.',
  "There's a sequel to a book written\ncalled <i>Butt Cake.</i>",
  SubRipTime(0, 0, 19, 986)),
 ('Ça marche bien dans les kiosques.\nUn cul plein de gâteau.',
  'Very popular at the news-stands.\nAbout a butt with cake all over it.',
  SubRipTime(0, 0, 20, 171)),
 ('<i>Gros culs</i>', '<i>Chunky Asses?</i>', SubRipTime(0, 0, 20, 743)),
 ('<i>Bienvenue au Journal Backman.\nJe suis Mel Backman.</i>',
  "<i>Welcome to the Mel Backman Journal.</i>\n<i>I'm Mel Backman.</i>",
  SubRipTime(0, 0, 23, 950)),
 ('<i>Je suis en compagnie\nde Chandler Jarrell...</i>',
  '<i>With me today is Chandler Jarrell</i>',
  SubRipTime(0, 0, 24, 163)),
 ('<i>dont l’action est très utile\nà la société.</i>',
  "<i>who's doing something</i>\n<i>very vital for the community.</i>",
  S

In [18]:
from googletrans import Translator

translator = Translator()

In [19]:
from functools import partial
from itertools import zip_longest
from operator import is_not
from tqdm import tqdm_notebook as tqdm

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks or blocks
    
    >>> " ".join(["".join(g) for g in grouper('ABCDEFG', 3, 'x')])
    'ABC DEF Gxx'
    """
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def grouper_without_fill(iterable, n):
    """Collect data into fixed-length chunks or blocks
    
    >>> " ".join(["".join(g) for g in grouper_without_fill('ABCDEFG', 3)])
    'ABC DEF G'
    """
    # https://stackoverflow.com/questions/16096754/remove-none-value-from-a-list-without-removing-the-0-value
    # https://stackoverflow.com/questions/40137950/possible-to-run-python-doctest-on-a-jupyter-cell-function
    # return map(lambda g: list(filter(lambda e: e is not None, g)), 
    return map(lambda g: filter(partial(is_not, None), g),
               grouper(iterable, n, fillvalue=None))

import doctest
doctest.run_docstring_examples(grouper, globals())

In [20]:
chunk_size = 15

# don't work ! :'( => not exactly ... 
# Seems google track IPs with "high" rate requests and bann (temporaly) IPs
for slice_srt_text in tqdm(
    map(lambda g: "#".join(g), 
        grouper(map(lambda srt: srt.text, srt_en), chunk_size, fillvalue="")),
    total=len(srt_en) // chunk_size
):
    try:
        t_slice_srt_text = translator.translate(slice_srt_text,
                                                src='en',
                                                dest='fr')
    except Exception as e:
        pass
    else:
        print(f"{t_slice_srt_text.text}")

Ne le laissez pas vous toucher! # <I> Butt Pie. </ I> # Il y a une suite à un livre écrit
appelé <i> Butt Cake. </ i> #Très populaire aux kiosques à journaux.
A propos d'un bout avec un gâteau partout. # <I> Chunky Asses? </ I> # <i> Bienvenue dans le journal Mel Backman. </ I>
<I> Je suis Mel Backman. </ I> # <i> Avec moi aujourd'hui, c'est Chandler Jarrell </ i> # <i> qui fait quelque chose </ i>
<I> très vital pour la communauté. </ i> # <i> Chandler est un découvreur d'enfants perdus. </ I>
<I> Ai-je raison? </ i> # - <i> Oui, je trouve des enfants disparus. </ I>
- <i> C'est très intéressant. </ I> # <i> Il y avait un spectacle appelé Finder </ i>
<I> of Lost Loves. Avez-vous vu cela? </ I> # - <i> Tony Franciosa était la star. </ I>
- <i> Ouais ... </ i> # - <i> Je cherche cette fille ... </ i>
- <i> J'espère que vous n'êtes pas nerveux. </ i> # <i> C'est parfaitement normal. C'est </ i>
<I> mon cinquième spectacle et je suis toujours nerveux. </ I> # <i> En fait, y a-t-il quelqu

Pas de problème ... je suis désolé,
Je veux juste des frites, c'est tout. # Tout ce que je veux, ce sont des frites. Une puce, ça y est. # Retournez ça, ça brûle. # Je veux juste des jetons. # Puis-je vous poser quelques questions? # - Ça devrait le retenir.
- Oui, ça devrait le retenir. # Hé, hé, hé! Dans ma poche,
il y a tout un tas de Tic Tacs. # Prenez-en autant que vous voulez. # Que faites-vous ici? #Hey! # Je vais casser votre cerveau, abruti! #Pourquoi êtes-vous venu ici?
Je vous ai dit de rester dans la voiture! # Je ne vais pas en faire une conférence, # mais la prochaine fois, je vous le dis
faire quelque chose, tu le fais. # Ceci est une affaire d'hommes. Ce n'est pas un endroit
pour une femme d'être, comprendre?
La prochaine fois que je te dis de faire quelque chose, fais-le.
Je ne veux pas que tu sois blessé. # - Puis-je gérer ça, OK?
- Ok. # Laissez-moi tranquille, connard. # Une fille tatouée avec un dragon jaune
était là. Qu'est-ce qui lui est arrivé? Quelle fille? # -

Vers 16h30,
mais je ne vais pas me rendre à la fête. # Salut. # Prends les choses en main. # Tu veux que j'achète un collier?
OK, je vais en acheter un. Je vais en acheter un.
Pouvez-vous me laisser le bleu? #Je ne sais pas
combien coûtent-ils, cependant? # Un de ceux-là? Ok. # Laissez moi le bleu,
ça correspond à mon manteau. # Attends une seconde, tu en as pris 100! #Non, un dollar, c'est pour toi.
Tu vois George Washington? # C'est Benjamin Franklin,
ça ne se passe pas. # Donnez ceci ... Vous avez le mauvais argent. # Puis-je récupérer mon argent? # - Vous m'avez aussi donné le mauvais collier.
- Tu me brises le coeur, asswipe # - Tu parles bien anglais.
- Alors toi, et alors?
Donne-moi mes 100 en retour, prends ce dollar
et donnez-moi le bon. # souffle de singe, vomir le visage,
mangeur de bave de tortue. # - Donne-moi ma merde et prends ta merde.
- Quoi, Scuzzbucket? #OOK, espèce de merde! # Tu es un petit bâtard sournois! #Tu veux ton argent? Prends-le! #Avez-vous vu un petit hom

J'ai honte de moi.
Je devrais être arrêté. # Je devrais être purgé,
Je devrais être fouetté. # - Je ne devrais pas marcher parmi les bonnes personnes.
- Laisse-moi lui parler. # Je suis un porc, un misérable.
Je ne mérite pas de vivre comme les autres. # Combien de temps pensez-vous
vous pouvez continuer cette mascarade? #Jusqu'à ce que je sois arrêté ou que vous réalisiez
les règles de preuve dans ce pays. # Si je suis arrêté, ils me prendront
et me mettre dans une cellule de prison. # Ils vont prendre le couteau, parce qu'il est volé,
et placez un autocollant «Pièce A» dessus. # Le couteau est assis dans une pièce
et je suis assis dans ma petite chambre jusqu'à ce que le procès arrive
un mois ou un an plus tard. # Si vous me faites arrêter, il est impossible de dire
quand tu auras ton couteau. # Tu n'as aucune idée de qui je suis, n'est-ce pas? Vous êtes Sardo Numsy! #Je me fous de qui vous êtes. je m'en préoccupe
que tu as kidnappé un gosse # Je pourrais te détruire comme ça.
Nous n

In [21]:
chunk_size = 15

# don't work ! :'( => not exactly ... 
# Seems google track IPs with "high" rate requests and bann (temporaly) IPs
for grp_srt_text in tqdm(grouper(map(lambda srt: srt.text, srt_en), chunk_size, fillvalue=""),
                           total=len(srt_en) // chunk_size):
    try:
        t_slice_srt_text = translator.translate(list(grp_srt_text), src='en', dest='fr')
    except Exception as e:
        pass
    else:
        pp.pprint(f"{list(map(lambda s: s.text, t_slice_srt_text))}")

("['Ne le laisse pas te toucher!', '<I> Butt Pie. </ I>', 'Il y a une suite à "
 "un livre écrit\\nappelé <i> Butt Cake. </ I>', 'Très populaire aux kiosques "
 "à journaux.\\nEnviron un bout avec un gâteau partout.', '<I> culs Chunky? </ "
 "i>', '<I> Bienvenue dans le Mel Backman Journal. </ I>\\n<I> Je suis Mel "
 'Backman. </ I>\', "<I> Avec moi aujourd\'hui est Chandler Jarrell </ i>", '
 "'<I> qui fait quelque chose </ i>\\n<I> très vital pour la communauté. </ "
 'I>\', "<I> Chandler est un chercheur d\'enfants perdus. </ I>\\n<I> Suis-je '
 'correct? </ I>", "- <i> Oui, je trouve des enfants disparus. </ I>\\n- <i> '
 'C\'est très intéressant. </ I>", \'<I> Il y avait un spectacle appelé Finder '
 "</ i>\\n<I> of Lost Loves. Avez-vous vu cela? </ I>', '- <i> Tony Franciosa "
 'était la star. </ I>\\n- <i> Ouais ... </ i>\', "- <I> je cherche cette '
 'fille ... </ i>\\n- <I> J\'espère que vous n\'êtes pas nerveux. </ I>", "<I> '
 "C'est parfaitement normal. C'est </ i>\\n<I> mo

('[\'Oui.\', "avez-vous d\'autres questions?", \'En fait je le fais.\', \'Que '
 "fais-tu ce week-end,\\nparce que ta silhouette est folle!', 'Elle joue aussi "
 'aux maraccas?\', "- C\'est l\'élu?\\n- Oui...", "Vous faites un bon '
 'spectacle.\\nOù l\'as-tu trouvée?", \'Elle est la bibliothécaire\\nau dépôt '
 "secret à Palkor Sin.', 'Elle a été volée ici pour nous aider.\\nElle a plus "
 "de 300 ans.', '- Comment a-t-elle géré celui-là?\\n- Un ancêtre a été violé "
 'par un dragon.\', "Ça arrive souvent\\nd\'où viens tu?", "Je suis désolé '
 'd\'avoir été un peu dur.\\nPourquoi ne montes-tu pas?", \'Non, merci.\', "- '
 'Qu\'Est-ce que c\'est?\\n- Les esprits, les esprits ...", \'Un esprit vient '
 "de me dire que\\ntu devrais arriver un peu,']")
('["prendre un verre,\\net attendez d\'autres instructions.", \'Je suis '
 "désolé. Pourquoi ne viens-tu pas\\npour un moment? Je serai bien.', "
 '"D\'ACCORD.", "J\'ai essayé.", \'Tu devrais toujours donner une femme\\ndeux '
 'chances de 

("['Avez-vous du méthyolate ou\\nmercurochrome, il pourrait être infecté?', "
 '\'Dis à tes amis que\\nSardo Numspa envoie ses salutations.\', "C\'est une '
 'erreur.", \'- Kee!\\n- Chandler!\', \'Ils vous ont ligoté dans du papier '
 'toilette.\', "- Que fais-tu ici?\\n- Je dois être ici, c\'est mon rêve.", '
 "'Ecoute, je pense\\nle gamin est ici quelque part.', '- On devrait trouver "
 "le gamin ensemble.\\n- Oui.', '- Après, on pourrait en dépenser ...\\n... du "
 "temps ensemble!', '- Oui, laisse la nature ...\\n... suit son cours.', "
 '\'Ouais.\', "Donc, c\'est Sardo Numspa.", "- C\'est quoi ce couteau?\\n- Le '
 'poignard croisé d\'Ajanti.", "Ils l\'ont apporté à ce monde\\npour tuer le '
 'deuxième enfant d\'or,", \'le porteur de la justice.\']')
('[\'Sa mort fut une grande perte.\', "Sardo en a besoin pour tuer '
 'l\'enfant,\\nmais vous ne pouvez l\'utiliser que pour le sauver.", "Vous '
 'devez obtenir le couteau et\\ninciter Numspa à libérer l\'enfant.", \'Vous '
 "ne deve

("['Espérons que le chemin reste sous moi!', '- Mais je devais rester sur le "
 "chemin!\\n- <i> Oui. </ i>', '<I> Mais vous devez savoir </ i>\\n<I> quand "
 "enfreindre les règles. </ I>', '- Combien ont survécu à ce test?\\n- <i> "
 'Aucun! </ i>\', \'Aucun?\', "Aucun? Je vais te casser le cul,\\nquand je '
 'sors d\'ici.", "Qu\'est-ce que je fais mal?", "J\'ai suivi toutes tes '
 'règles,\\nQu\'est-ce que je fais mal?", "Gardez vos pensées aussi '
 'pures\\ncomme l\'eau et ne laissez pas tomber l\'eau.", "Ne laisse pas '
 'tomber l\'eau.", \'Attends une minute, attends une minute ...\', '
 '"L\'eau...\\nLe feu et l\'eau ...", "D\'ACCORD.", \'À Monty!\', \'Un '
 "toast!']")
('["J\'ai le couteau,\\nmaintenant allume la foutue lumière!", "Qu\'est-ce que '
 'c\'est, asswipe?", \'Je veux demander votre avis\\nà propos de quelque '
 'chose.\', "Je vais laisser cette diapositive,\\nparce que c\'est ton '
 'chemin.", "N\'appuyez pas sur votre chance, cependant.", \'Je sais que tu '
 "es\\n

In [22]:
import urllib.request
from urllib.parse import quote
import sys

typ = sys.getfilesystemencoding()

def translate(querystr, to_l="zh", from_l="en"):
    '''for google tranlate by doom
    '''
    C_agent = {'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.165063 Safari/537.36 AppEngine-Google."}
    flag = 'class="t0">'
    tarurl = ("http://translate.google.cn/m?hl=%s&sl=%s&q=%s" % (to_l, from_l, quote(querystr, safe='')))
    request = urllib.request.Request(tarurl, headers=C_agent)
    page = str(urllib.request.urlopen(request).read().decode(typ))
    target = page[page.find(flag) + len(flag):]
    target = target.split("<")[0]
    return target

In [23]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

cleanr = re.compile('<.*?>')

In [24]:
from urllib.parse import unquote
from html import unescape  # https://docs.python.org/3/library/html.html

pp = pprint.PrettyPrinter(indent=4)

# big chunk
# TODO: need to find the maximum size allowed to transfer
# Pb: hard to control the seperation (and reconstruction) of blob text for translation
# ... google translate seems to do strange things (unification, interpretation) with the split_token ...
# This behavious disturbed the translation and introduce offsets on (block of) translations
# reduce chuck to unit => very slow but working :-/
chunk_size = 1
# TODO: Trying to find a better split-token
split_token = ' ¶ '

gen_slice_srt_text = map(
    lambda g: split_token.join(map(lambda s: ''.join(cleanhtml(s)), g)),
    grouper_without_fill(map(lambda srt: srt.text, srt_en), chunk_size)
)
for slice_srt_text in tqdm(gen_slice_srt_text,
                           total=len(srt_en) // chunk_size):
    try:
        t_slice_srt_text = unescape(translate(slice_srt_text, from_l='en', to_l='fr'))
    except Exception as e:
        break
    else:
        # https://stackoverflow.com/questions/44780357/how-to-use-newline-n-in-f-string-to-format-output-in-python-3-6
#         t_srt_text = "\n¶ ".join(t_slice_srt_text.split('¶'))
#         print(f"{t_srt_text}")
        pp.pprint(list(zip(slice_srt_text.split(split_token), t_slice_srt_text.split(split_token))))


[("Don't let him touch you!", 'Ne le laisse pas te toucher!')]
[('Butt Pie.', 'Butt Pie.')]
[   (   "There's a sequel to a book written\ncalled Butt Cake.",
        'Il y a une suite à un livre écrit appelé Butt Cake.')]
[   (   'Very popular at the news-stands.\nAbout a butt with cake all over it.',
        'Très populaire aux kiosques à journaux. Environ un bout avec un '
        'gâteau partout.')]
[('Chunky Asses?', 'Culs Chunky?')]
[   (   "Welcome to the Mel Backman Journal.\nI'm Mel Backman.",
        'Bienvenue dans le Mel Backman Journal. Je suis Mel Backman.')]
[   (   'With me today is Chandler Jarrell',
        "Chandler Jarrell est avec moi aujourd'hui")]
[   (   "who's doing something\nvery vital for the community.",
        'qui fait quelque chose très vital pour la communauté.')]
[   (   'Chandler is a finder of lost children.\nAm I correct?',
        "Chandler est un chercheur d'enfants perdus. Ai-je raison?")]
[   (   "- Yes, I find missing children.\n- That's very in

[('Lick it and smoke it.', 'Lèche-le et fume-le.')]
[   (   'I think there were two of these,\nand you smoked the other one.',
        "Je pense qu'il y en avait deux, et vous avez fumé l'autre.")]
[   (   'Put it in your bag\nand stop smoking scrolls.',
        'Le mettre dans votre sac et arrêter de fumer des parchemins.')]
[   (   'It looks like a joint. You might get\n'
        'jumped by Rastafarians, so put it away.',
        'Cela ressemble à un joint. Vous pourriez obtenir sauté par les '
        'Rastafariens, alors rangez-le.')]
[   (   'Excuse me, I have a game to play.\nIt was a nice thought.',
        "Excusez-moi, j'ai un jeu à jouer. C'était une belle pensée.")]
[   (   "I realise it's hard for you to take in,",
        "Je me rends compte qu'il est difficile pour vous de prendre en "
        'compte,')]
[   (   "but it's your destiny\nto find the Golden Child.",
        "mais c'est ton destin pour trouver l'enfant d'or.")]
[   (   "And it's your destiny\nto seek psychia

[('She plays the maraccas, too?', 'Elle joue aussi aux maraccas?')]
[('- This is the Chosen One?\n- Yes...', "- C'est l'élu? - Oui...")]
[   (   'You people put on a good show.\nWhere did you find her?',
        "Vous faites un bon spectacle. Où l'as-tu trouvée?")]
[   (   "She's the librarian\nat the Secret Repository at Palkor Sin.",
        'Elle est la bibliothécaire au dépôt secret à Palkor Sin.')]
[   (   "She was flown here to help us.\nShe's over 300 years old.",
        'Elle a été volée ici pour nous aider. Elle a plus de 300 ans.')]
[   (   '- How did she manage that one?\n- An ancestor was raped by a dragon.',
        '- Comment a-t-elle géré celui-là? - Un ancêtre a été violé par un '
        'dragon.')]
[   (   "Does that happen a lot\nwhere you're from?",
        "Ça arrive souvent d'où viens tu?")]
[   (   "I'm sorry I got a little harsh.\nWhy don't you come up?",
        "Je suis désolé d'avoir été un peu dur. Pourquoi ne montes-tu pas?")]
[('No, thank you.', 'Non, mer

[   (   'Stop acting stupid,\nthe man owns the place, all right?',
        "Arrête d'être stupide, l'homme possède la place, d'accord?")]
[("I'm looking for Tommy Tong.", 'Je cherche Tommy Tong.')]
[   (   "You don't speak English?\nLet me translate it.",
        'Tu ne parles pas anglais? Laissez-moi le traduire.')]
[   (   "Bring Tommy Tong here\nor I'll bust your ass.",
        'Amenez Tommy Tong ici ou je te casserai le cul.')]
[('You understand that?', 'Tu comprends ça?')]
[   (   'You see this dead girl? The last time\n'
        'she was seen alive was with Tong.',
        'Tu vois cette fille morte? La dernière fois elle a été vue en vie '
        'était avec Tong.')]
[   (   "Tell Mr Tong that he has a guest\nor I'll bust your ass!",
        "Dites à M. Tong qu'il a un invité ou je te casse le cul!")]
[   (   'Let me show you\nhow I feel about that karate shit, OK?',
        'Laisse moi te montrer ce que je ressens à propos de cette merde de '
        "karaté, d'accord?")]
[   

[   (   "They've tied you up in toilet paper.",
        'Ils vous ont ligoté dans du papier toilette.')]
[   (   "- What are you doing here?\n- I have to be here, it's my dream.",
        "- Que fais-tu ici? - Je dois être ici, c'est mon rêve.")]
[   (   'Listen, I think\nthe kid is here somewhere.',
        'Ecoute, je pense le gamin est ici quelque part.')]
[   (   '- We should find the kid together.\n- Yes.',
        '- On devrait trouver le gamin ensemble. - Oui.')]
[   (   '- Afterwards, we could spend some...\n...time together!',
        '- Après, on pourrait en dépenser ... ... du temps ensemble!')]
[   (   '- Yes, kind of let nature...\n...take its course.',
        '- Oui, laisse la nature ... ... suit son cours.')]
[('Yeah.', 'Ouais.')]
[('So, it is Sardo Numspa.', "Donc, c'est Sardo Numspa.")]
[   (   "- What's this knife?\n- The Crossed Dagger of Ajanti.",
        "- C'est quoi ce couteau? - Le poignard croisé d'Ajanti.")]
[   (   'They brought it to this world\nto kill the

[   (   "Of course you are. I'll arrange\nfor our trek into the mountains.",
        'Bien sûr, vous êtes. Je vais arranger pour notre trek dans les '
        'montagnes.')]
[   (   'You have about an hour to look around.',
        'Vous avez environ une heure pour regarder autour de vous.')]
[   (   "About 16:30,\nbut I won't make it to the party.",
        'Vers 16h30, mais je ne vais pas me rendre à la fête.')]
[('Hi.', 'Salut.')]
[('Take it easy.', 'Relax.')]
[   (   "You want me to buy a necklace?\nOK, I'll buy one.",
        "Tu veux que j'achète un collier? OK, je vais en acheter un.")]
[   (   "I'll buy one.\nCan you let me have the blue one?",
        'Je vais en acheter un. Pouvez-vous me laisser le bleu?')]
[   (   "I don't know\nhow much they cost, though.",
        'Je ne sais pas combien ils coûtent, cependant.')]
[('One of those? OK.', "L'un de ces? D'ACCORD.")]
[   (   'Let me have the blue one,\nit matches my coat.',
        'Laisse-moi le bleu, ça va avec mon manteau.

[   (   "Let's just hope the path stays under me!",
        'Espérons que le chemin reste sous moi!')]
[   (   '- But I was supposed to stay on the path!\n- Yes.',
        '- Mais je devais rester sur le chemin! - Oui.')]
[   (   'But you must know\nwhen to break the rules.',
        'Mais tu dois savoir quand enfreindre les règles.')]
[   (   '- How many have survived this test?\n- None!',
        '- Combien ont survécu à ce test? - Aucun!')]
[('None?', 'Aucun?')]
[   (   "None? I'm going to break your ass,\nwhen I get out of here.",
        "Aucun? Je vais te casser le cul, quand je sors d'ici.")]
[('What am I doing wrong?', "Qu'est-ce que je fais mal?")]
[   (   'I followed all your rules,\nwhat am I doing wrong?',
        "J'ai suivi toutes tes règles, Qu'est-ce que je fais mal?")]
[   (   "Keep your thoughts as pure\nas the water and don't drop the water.",
        "Gardez vos pensées aussi pures comme l'eau et ne laissez pas tomber "
        "l'eau.")]
[("Don't drop the water.", 

[   (   'It just does my heart good\nto see another American citizen.',
        'Ça fait du bien à mon coeur voir un autre citoyen américain.')]
[('- Hold it!\n- Are you crazy?', '- Le tenir! - Êtes-vous fou?')]
[   (   "No, I'm Agent Jarrell from the American\nStolen Artefacts Foundation.",
        "Non, je suis l'agent Jarrell de l'américain Fondation d'artefacts "
        'volés.')]
[   (   "Everybody, it's OK! I'm with\nthe Stolen Artefacts Foundation!",
        'Tout le monde, ça va! je suis avec la Fondation des objets volés!')]
[   (   'Freddie, get out of my sight!\nYou make me sick!',
        'Freddie, sors de ma vue! Vous me dégoûtez!')]
[   (   "I'll talk to you on the plane.\nNobody be alarmed!",
        "Je te parlerai dans l'avion. Personne ne soit alarmé!")]
[   (   "I'm Chandler Jarrell,\nAmerican Stolen Artefacts Foundation.",
        'Je suis Chandler Jarrell, Fondation américaine des artefacts volés.')]
[   (   'Thanks to you, we were able\nto catch this man, Freddie

[   (   'You are the Chosen One.\nYou will find the child.',
        "Vous êtes l'élu. Vous trouverez l'enfant.")]
[   (   'You have until nightfall to find him.\n'
        "Sardo can't kill him until dark.",
        "Vous avez jusqu'à la nuit pour le trouver. Sardo ne peut pas le tuer "
        'avant la nuit.')]
[("We'll get you out of here, OK?", "On va te sortir d'ici, d'accord?")]
[("Where's your boss?", 'Où est ton patron?')]
[   (   "Keep an eye on the kid.\nDon't worry, I'm the Chosen One.",
        "Gardez un œil sur l'enfant. Ne vous inquiétez pas, je suis l'élu.")]
[("This won't take long.", 'Cela ne prendra pas longtemps.')]
[('My dear, sweet brother Numsy!', 'Mon cher et gentil frère Numsy!')]
[   (   "I can see that you're busy right now.\n"
        "I'll come back some other time.",
        'Je peux voir que vous êtes occupé en ce moment. Je reviendrai une '
        'autre fois.')]
[('There are no keys in the car!', "Il n'y a pas de clés dans la voiture!")]
[('Come on, c

In [ ]:
from urllib.parse import unquote
from html import unescape  # https://docs.python.org/3/library/html.html

pp = pprint.PrettyPrinter(indent=4)

from_l = 'en'
to_l = 'fr'

def gen_translate_srt(srt_from_l):
    for srt_from_l in tqdm(srt_from_l):
        try:
            srt_txt_from_l = cleanhtml(srt_translated['srt_from_l'].text).replace('\n', ' ')
            srt_txt_to_l = unescape(translate(srt_txt_from_l, from_l=from_l, to_l=to_l))
        except Exception as e:
            break
        else:
            yield {
                'srt_from_l': srt_from_l,
                'srt_txt_from_l': srt_txt_from_l,
                'google_translate_to_l': srt_txt_to_l
            }

            
srts_translated = []
for srt_translated in gen_translate_srt(srt_en):        
    srts_translated.append(srt_translated)
    pp.pprint(
        (
            srt_translated['srt_from_l'].index,
            srt_translated['srt_txt_from_l'],
            srt_translated['google_translate_to_l']
        )
    )
